# Hyper-parameter tutning: XGBRegressor

In [0]:
# Loading libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# setting paths
data_dir = os.path.abspath('/content/drive/My Drive/Projects/YapAiTek_Challenge/_data')
train_data_fp = os.path.join(data_dir, 'X_train_final.npy')
test_data_fp = os.path.join(data_dir, 'X_test_final.npy')
y_true_fp = os.path.join(data_dir, 'y_train_final.npy')

In [0]:
# Loading data-sets
train = np.load(train_data_fp)
target = np.load(y_true_fp)
# test = np.load(test_data_fp)

In [0]:
rs = 13  # random_state constant
cv = 3   # kfold constant

In [0]:
def evaluate(actual, prediction):
    return {'MAE': round(mean_absolute_error(actual, prediction), 5),
            'MSE': round(mean_squared_error(actual, prediction), 5),
            'R^2': round(r2_score(actual, prediction), 5)}

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.3)

## Hyper-parameter grid search for XGBRegressor estimator

In [0]:
xgb_regressor = XGBRegressor()

xgb_params = {
    'n_estimators': [30, 150, 250],
    'max_depth': [3, 6, 15],
    'learning_rate': [0.1, 0.01, 0.001]
}

In [0]:
%%time

xg_gs = GridSearchCV(estimator=xgb_regressor, 
                     param_grid=xgb_params, 
                     scoring='neg_mean_absolute_error', 
                     cv=cv, 
                     n_jobs=-1)

xg_gs_fit = xg_gs.fit(train, target)

xg_gs_df = pd.DataFrame(xg_gs_fit.cv_results_)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


[08:28:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 2min 37s, sys: 684 ms, total: 2min 38s
Wall time: 3h 2min 6s


In [0]:
xg_gs_df.sort_values(by='mean_test_score', ascending=False)[:10]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
4,165.245242,0.574539,2.334585,0.047487,0.1,6,150,"{'learning_rate': 0.1, 'max_depth': 6, 'n_esti...",-1.564276,-1.481036,-1.618500,-1.554604,0.056535,1
2,128.593879,1.200713,1.874844,0.012682,0.1,3,250,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1.546276,-1.547643,-1.630067,-1.574662,0.039181,2
3,34.849478,0.154291,0.426028,0.007838,0.1,6,30,"{'learning_rate': 0.1, 'max_depth': 6, 'n_esti...",-1.571957,-1.560466,-1.607905,-1.580109,0.020207,3
17,951.417484,9.627492,19.168889,0.726341,0.01,15,250,"{'learning_rate': 0.01, 'max_depth': 15, 'n_es...",-1.650430,-1.490060,-1.610624,-1.583705,0.068182,4
1,77.541420,0.388452,1.141194,0.017793,0.1,3,150,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1.560653,-1.570483,-1.647707,-1.592948,0.038928,5
6,114.235621,0.782274,1.322721,0.019799,0.1,15,30,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",-1.676487,-1.480622,-1.625877,-1.594328,0.083015,6
14,280.662397,4.223460,3.004203,0.090574,0.01,6,250,"{'learning_rate': 0.01, 'max_depth': 6, 'n_est...",-1.587740,-1.583610,-1.620575,-1.597309,0.016538,7
16,572.695876,5.164845,9.349979,0.729976,0.01,15,150,"{'learning_rate': 0.01, 'max_depth': 15, 'n_es...",-1.686978,-1.564668,-1.594797,-1.615481,0.052031,8
13,170.241899,3.354943,1.837495,0.041906,0.01,6,150,"{'learning_rate': 0.01, 'max_depth': 6, 'n_est...",-1.653236,-1.660046,-1.644806,-1.652696,0.006233,9
5,272.007654,0.940201,3.952049,0.077640,0.1,6,250,"{'learning_rate': 0.1, 'max_depth': 6, 'n_esti...",-1.589440,-1.473561,-1.910261,-1.657754,0.184710,10


## Test various hyper-parameters for XGBRegressor estimator

In [0]:
# %%time

# xgb = XGBRegressor(n_estimators=150, max_depth=6, learning_rate=0.1)
# xgb.fit(X_train, y_train)
# preds = xgb.predict(X_test)
# print(evaluate(y_test, preds))

[08:41:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
{'MAE': 1.43698, 'MSE': 5.68024, 'R^2': 0.76444}
CPU times: user 1min 56s, sys: 132 ms, total: 1min 56s
Wall time: 1min 56s


In [0]:
# %%time

# xgb = XGBRegressor(n_estimators=150, max_depth=6, learning_rate=0.01)
# xgb.fit(X_train, y_train)
# preds = xgb.predict(X_test)
# print(evaluate(y_test, preds))

[09:05:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
{'MAE': 1.6285, 'MSE': 8.2654, 'R^2': 0.65723}
CPU times: user 1min 56s, sys: 134 ms, total: 1min 56s
Wall time: 1min 56s


In [0]:
# %%time

# xgb = XGBRegressor(n_estimators=150, max_depth=6, learning_rate=0.5, objective='reg:squarederror')
# xgb.fit(X_train, y_train)
# preds = xgb.predict(X_test)
# print(evaluate(y_test, preds))

{'MAE': 1.38925, 'MSE': 5.3139, 'R^2': 0.77963}
CPU times: user 1min 54s, sys: 270 ms, total: 1min 55s
Wall time: 1min 55s


In [0]:
# %%time

# xgb = XGBRegressor(n_estimators=150, max_depth=15, learning_rate=0.5, objective='reg:squarederror')
# xgb.fit(X_train, y_train)
# preds = xgb.predict(X_test)
# print(evaluate(y_test, preds))

{'MAE': 1.3719, 'MSE': 5.49201, 'R^2': 0.77225}
CPU times: user 6min 39s, sys: 472 ms, total: 6min 40s
Wall time: 6min 40s


In [8]:
# %%time

# xgb = XGBRegressor(n_estimators=150, max_depth=12, learning_rate=0.55, objective='reg:squarederror')
# xgb.fit(X_train, y_train)
# preds = xgb.predict(X_test)
# print(evaluate(y_test, preds))

{'MAE': 1.37822, 'MSE': 5.42462, 'R^2': 0.77678}
CPU times: user 4min 27s, sys: 527 ms, total: 4min 28s
Wall time: 4min 29s


### Best hyper-parameters: XGBRegressor

In [0]:
%%time

xgb = XGBRegressor(n_estimators=150, max_depth=10, learning_rate=0.5, objective='reg:squarederror')
xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)
print(evaluate(y_test, preds))

{'MAE': 1.36346, 'MSE': 5.21117, 'R^2': 0.78389}
CPU times: user 3min 46s, sys: 271 ms, total: 3min 46s
Wall time: 3min 47s
